In [10]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
import pandas as pd
import numpy as np

import gzip, pickle

### 1. Loading MNIST data

[MNIST Dataset](#http://deeplearning.net/data/mnist/mnist.pkl.gz)

In [5]:
with gzip.open('mnist.pkl.gz','rb') as ff :
    u = pickle._Unpickler( ff )
    u.encoding = 'latin1'
    train, val, test = u.load()

In [6]:
print( train[0].shape, train[1].shape )

(50000, 784) (50000,)


In [7]:
print( val[0].shape, val[1].shape )

(10000, 784) (10000,)


In [8]:
print( test[0].shape, test[1].shape )

(10000, 784) (10000,)


In [ ]:
some_digit = train[0][0]
some_digit_image = some_digit.reshape(28, 28)
plt.imshow(some_digit_image, cmap = mpl.cm.binary, interpolation="nearest")
plt.axis("off")
plt.show()

In [13]:
train[1][0]

5

In [ ]:
train[1] = train[1].astype(np.uint8)

### 2. Train a Binary Classifier